In [1]:
from keras import __version__
import numpy as np
from keras.models import model_from_json, Model
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

Using TensorFlow backend.


In [66]:
weights_path = 'inceptionv3_clothing_classifier_first.h5'
json_path = 'incep_filter_clothing_classifier.json'

json_file = open(json_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights(weights_path)
print("Loaded model from disk")

# evaluate loaded model on test data
# loaded_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
#score = loaded_model.evaluate(X, Y, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk


In [67]:
def image_preprocess(img_path):
    image = load_img(img_path, target_size=(299, 299))
    image = img_to_array(image)

    # our input image is now represented as a NumPy array of shape
    # (inputShape[0], inputShape[1], 3) however we need to expand the
    # dimension by making the shape (1, inputShape[0], inputShape[1], 3)
    # so we can pass it through thenetwork
    image = np.expand_dims(image, axis=0)

    # pre-process the image using the appropriate function based on the
    # model that has been loaded (i.e., mean subtraction, scaling, etc.)
    image = preprocess_input(image)
    return image

In [87]:
clothes_labels = ['dresses', 'jackets', 'jeans', 'shorts', 'skirts'  
                  'sweaters', 'sweatshirts', 'womens-outerwear',
                  'womens-pants', 'womens-tops']

def get_image_prediction(model, img_path):
    img = image_preprocess(img_path)
    preds = loaded_model.predict(img)[0]
    preds_labels = list(zip(clothes_labels, preds))
    preds_labels.sort(key=lambda p: p[1], reverse=True)
    return preds_labels

jeans_img_path = 'data/validation/jeans/666416190.jpg'
dress_img_path = 'data/validation/dresses/535993294.jpg'
jackets_img_path = 'data/validation/jackets/614521319.jpg'
jackets_img_path2 = 'data/validation/jackets/614763686.jpg'
get_image_prediction(loaded_model, dress_img_path)

[('dresses', 0.97745091),
 ('womens-pants', 0.0051303385),
 ('womens-outerwear', 0.0033489601),
 ('womens-tops', 0.0024949466),
 ('jackets', 0.001503865),
 ('sweatshirts', 0.00056735112),
 ('skirtssweaters', 0.0004758416),
 ('jeans', 0.00037440032),
 ('shorts', 0.00024481604)]

In [69]:
loaded_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [70]:
loaded_model.layers.pop() # Get rid of the classification layer

In [71]:
last = loaded_model.layers[-1].output
model = Model(loaded_model.input, last)
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, None, None, 32 96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, None, None, 32 0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [96]:
dog_img_path = 'data-pets/train/dogs/dog.0.jpg'

img_jacket = image_preprocess(jackets_img_path)
img_jacket2 = image_preprocess(jackets_img_path2)
img_jeans = image_preprocess(jeans_img_path)
img_dress = image_preprocess(dress_img_path)
img_dog = image_preprocess(dog_img_path)

jacket_feat = model.predict(img_jacket)
jacket_feat2 = model.predict(img_jacket2)
jeans_feat = model.predict(img_jeans)
dress_feat = model.predict(img_dress)
dog_feat = model.predict(img_dog)

In [98]:
# compare cosine similarities
print('2 jackets: ', cosine_similarity(jacket_feat, jacket_feat2))
print('jeans and dress: ', cosine_similarity(jeans_feat, dress_feat))
print('jeans and dog: ', cosine_similarity(jeans_feat, dog_feat))

2 jackets:  [[ 0.87895989]]
jeans and dress:  [[ 0.85998255]]
jeans and dog:  [[ 0.8234235]]


## Simple Pairing Model

In [7]:
def get_image_pairs(filename, dir_path, y_output):
    valid_pairs = []
    with open(filename, 'r') as f:
        for line in f.readlines()[:5]:
            l = line.split()
            pair_id = l[0]
            product1_id = l[2]
            product2_id = l[5]
            product1_path = dir_path + pair_id + '_' + product1_id + '.jpg' 
            product2_path = dir_path + pair_id + '_' + product2_id + '.jpg' 
            valid_pairs.append((product1_path, product2_path, y_output))

    return valid_pairs

get_image_pairs('data-pairs/small_valid_clothing_pairs.tsv', 'data-pairs/valid/', 1)
get_image_pairs('data-pairs/small_invalid_clothing_pairs.tsv', 'data-pairs/invalid/', 0)

[('data-pairs/valid/17973_620987510.jpg',
  'data-pairs/valid/17973_643055627.jpg',
  1),
 ('data-pairs/valid/1523_327428339.jpg',
  'data-pairs/valid/1523_625092820.jpg',
  1),
 ('data-pairs/valid/17712_533288681.jpg',
  'data-pairs/valid/17712_542767398.jpg',
  1),
 ('data-pairs/valid/16662_525034002.jpg',
  'data-pairs/valid/16662_637073662.jpg',
  1),
 ('data-pairs/valid/8596_639124928.jpg',
  'data-pairs/valid/8596_616568745.jpg',
  1),
 ('data-pairs/valid/6956_602530825.jpg',
  'data-pairs/valid/6956_610124283.jpg',
  1),
 ('data-pairs/valid/8323_648321071.jpg',
  'data-pairs/valid/8323_602760991.jpg',
  1),
 ('data-pairs/valid/4026_656878604.jpg',
  'data-pairs/valid/4026_631524942.jpg',
  1),
 ('data-pairs/valid/1603_638938070.jpg',
  'data-pairs/valid/1603_626907621.jpg',
  1),
 ('data-pairs/valid/5041_472267824.jpg',
  'data-pairs/valid/5041_626675431.jpg',
  1),
 ('data-pairs/valid/16470_606112017.jpg',
  'data-pairs/valid/16470_623558973.jpg',
  1),
 ('data-pairs/valid/3487